# Import Data

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style(style='whitegrid')
import matplotlib.pyplot as plt
import os, re, operator, warnings
warnings.filterwarnings('ignore')  # Let's not pay heed to them right now

In [3]:
#sets the default options for viewing pandas dataframes
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)
#pd.set_option('display.width', 100)
pd.set_option('display.max_info_columns', 50)

In [4]:
path = '/Users/dmitriykats/Documents/SpringBoard/Springboard/Capstone2/true_review/data/'

In [5]:
df = pd.read_csv(f'{path}/raw/restaurant_reviews.csv', parse_dates=['date'])

In [6]:
df.columns

Index(['user_id', 'business_id', 'rev_stars', 'date', 'text', 'useful',
       'funny', 'cool', 'bus_id', 'name', 'neighborhood', 'address', 'city',
       'state', 'postal_code', 'latitude', 'longitude', 'bus_stars',
       'review_count', 'is_open', 'categories'],
      dtype='object')

In [7]:
#drop the bus_id columns, it's a duplicated of business_id
df = df.drop(columns='bus_id')

In [8]:
df['weekday'] = df.date.apply(lambda x: x.weekday())

In [9]:
#add a new colomn which calculates the number of characters in a given review
df['text length'] = df['text'].apply(len)

In [10]:
#add a year column
df['year'] = df.date.apply(lambda x: x.year)

In [11]:
scottsdale = df[df.city == 'Scottsdale']

In [12]:
print(f'Number of Users: {scottsdale.user_id.unique().shape[0]}')
print(f'Number of Restaurants: {scottsdale.business_id.unique().shape[0]}')
print(f'Number of Reviews: {scottsdale.text.unique().shape[0]}')

Number of Users: 76011
Number of Restaurants: 1322
Number of Reviews: 173062


In [13]:
stops = ['Restaurants', 'Food', 'Nightlife']
scottsdale['split_categories'] = scottsdale.categories.apply(lambda x: x.split(';'))
scottsdale['string_categories'] = scottsdale.split_categories.apply(lambda x: ' '.join(x))
scottsdale.to_csv('scott_data.csv', encoding='utf-8')

### Looking at Checkins

In [14]:
df_check = pd.read_csv(f'{path}/external/yelp_checkin.csv')

In [15]:
df_check.head()

,business_id,weekday,hour,checkins
0,3Mc-LxcqeguOXOVT_2ZtCg,Tue,0:00,12
1,SVFx6_epO22bZTZnKwlX7g,Wed,0:00,4
2,vW9aLivd4-IorAfStzsHww,Tue,14:00,1
3,tEzxhauTQddACyqdJ0OPEQ,Fri,19:00,1
4,CEyZU32P-vtMhgqRCaXzMA,Tue,17:00,1


### Users and their friends

In [16]:
df_friends = pd.read_csv(f'{path}/external/yelp_user.csv', parse_dates=['yelping_since'])

In [17]:
df_friends.head()

,user_id,name,review_count,yelping_since,friends,useful,funny,cool,fans,elite,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,JJ-aSuM4pCFPdkfoZ34q0Q,Chris,10,2013-09-24,"0njfJmB-7n84DlIgUByCNw, rFn3Xe3RqHxRSxWOU19Gpg...",0,0,0,0,None,3.70,0,0,0,0,0,0,0,0,0,0,0
1,uUzsFQn_6cXDh6rPNGbIFA,Tiffy,1,2017-03-02,None,0,0,0,0,None,2.00,0,0,0,0,0,0,0,0,0,0,0
2,mBneaEEH5EMyxaVyqS-72A,Mark,6,2015-03-13,None,0,0,0,0,None,4.67,0,0,0,0,0,0,0,0,0,0,0
3,W5mJGs-dcDWRGEhAzUYtoA,Evelyn,3,2016-09-08,None,0,0,0,0,None,4.67,0,0,0,0,0,0,0,0,0,0,0
4,4E8--zUZO1Rr1IBK4_83fg,Lisa,11,2012-07-16,None,4,0,0,0,None,3.45,0,0,0,0,0,0,0,0,0,1,0


In [18]:
df_friends.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1326100 entries, 0 to 1326099
Data columns (total 22 columns):
user_id               1326100 non-null object
name                  1325603 non-null object
review_count          1326100 non-null int64
yelping_since         1326100 non-null datetime64[ns]
friends               1326100 non-null object
useful                1326100 non-null int64
funny                 1326100 non-null int64
cool                  1326100 non-null int64
fans                  1326100 non-null int64
elite                 1326100 non-null object
average_stars         1326100 non-null float64
compliment_hot        1326100 non-null int64
compliment_more       1326100 non-null int64
compliment_profile    1326100 non-null int64
compliment_cute       1326100 non-null int64
compliment_list       1326100 non-null int64
compliment_note       1326100 non-null int64
compliment_plain      1326100 non-null int64
compliment_cool       1326100 non-null int64
compliment_funny 

In [19]:
#filter the friends dataframe to only include users with friends
df_friends['friends'] = df_friends.friends.apply(lambda x: np.nan if x == 'None' else x)
df_friends = df_friends.dropna()

In [20]:
df_friends.head()

,user_id,name,review_count,yelping_since,friends,useful,funny,cool,fans,elite,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,JJ-aSuM4pCFPdkfoZ34q0Q,Chris,10,2013-09-24,"0njfJmB-7n84DlIgUByCNw, rFn3Xe3RqHxRSxWOU19Gpg...",0,0,0,0,None,3.70,0,0,0,0,0,0,0,0,0,0,0
17,h5ERTYn2vQ1QbjTZvfWPaA,Bobby,3,2017-01-07,"jYiZnueCr7gVq9T34xoa7g, yFLXGdY6rpHt7hRiwEFMag...",0,0,0,0,None,3.33,0,0,0,0,0,0,0,0,0,0,0
18,jYnkJR3T8yCERXywoVhWYA,Hugo,48,2010-07-06,"hkXekeW_Jj6mIy8r8N7r1Q, dQDpV-VUtwYGqHznuRV-yw...",15,6,2,3,None,3.73,2,0,0,0,0,1,1,1,1,2,0
44,fV8Yr0c5tFQTQ2SRRJHXHw,Michelle,50,2007-08-22,"HDb4fBWIAQ-foS8qLJty9w, x0hBZsmBTYxhjjx0MShz1A...",43,12,5,1,2009,3.96,2,4,0,0,0,9,7,7,7,4,0
74,aw973Pm1nrTbRjP4zY9B9g,Kenny,762,2008-09-23,"Cit5yho-DqotA0BnXHErTQ, bm2DqfP4P454FjEtCbZdkQ...",174,151,67,27,"2014, 2016, 2013, 2015, 2010, 2012, 2011",3.60,20,4,2,7,0,16,52,47,47,14,1


In [21]:
#Further filter the dataframe to only include users with over 200 reviews
df_power_user = df_friends[df_friends.review_count > 200]

In [22]:
df_power_user['num_friends'] = df_power_user.friends.apply(lambda x: x.count(',') + 1)

In [23]:
df_power_user['year'] = df_power_user.yelping_since.apply(lambda x: x.year)

In [24]:
#list of users with over 200 reviews and friends
power_user = list(df_power_user.user_id.unique())

### Dataframe for Scottsdale, with power users only

In [25]:
df.head()

,user_id,business_id,rev_stars,date,text,useful,funny,cool,name,neighborhood,address,city,state,postal_code,latitude,longitude,bus_stars,review_count,is_open,categories,weekday,text length,year
0,DUfWxxewcJgGcc1Y189XSQ,ZcAoKSDT6BNcWsn3o4-WVA,2,2017-01-07,Cool place packed to the rafters. Menu is pret...,1,0,0,"""Me & Julio""",NaN,"""2784 S Fish Hatchery Rd""",Fitchburg,WI,53711,43.005728,-89.427154,3.0,102,1,Restaurants;Mexican;Tex-Mex;American (Traditio...,5,369,2017
1,oAJf-_URH4QBMs2ZMpjUyA,9VhZRkmyoEho58vmnMh_Hg,3,2016-04-11,"I've had mixed experiences at this Montana's, ...",0,0,0,"""Montana's BBQ & Bar""",NaN,"""8735 Yonge St""",Richmond Hill,ON,L4C 6Z1,43.840823,-79.428224,3.0,49,1,Comfort Food;Restaurants;American (Traditional...,0,524,2016
2,SOb8RgcqBGbwlq9LZXy3fQ,IgOaTag1zhIXndxngP0M3w,1,2016-09-21,Die Angaben hier sind leider nicht mehr ganz k...,0,0,0,"""Restaurant Ganesha""",NaN,"""Auberlenstr. 40""",Fellbach,BW,70736,48.818648,9.270925,3.0,8,1,Restaurants;Indian,2,125,2016
3,Iwe-xN6dnugdZ0KiQjO6Hg,Fvd8qeJCXFbMYQk45S7iBQ,5,2014-06-30,My family came in while waiting for our time a...,0,0,0,"""Las Fuentes Mexican Grill""",NaN,"""13621 N Litchfield Rd""",Surprise,AZ,85374,33.607472,-112.355941,3.5,151,1,Mexican;Restaurants,0,256,2014
4,VSCpdfEtjdWjTbBVKMb5eA,mF2EW3twSrFPmT_RVV1-Qg,2,2011-11-18,I would not recommend this place for Chinese f...,3,5,0,"""House of Hunan""",NaN,"""18 Public Sq""",Medina,OH,44256,41.138544,-81.864299,3.0,83,1,Restaurants;Chinese,4,992,2011


In [26]:
sdl_df = df[(df.city == 'Scottsdale') & (df.user_id.isin(power_user))]

In [27]:
sdl_df.head()

,user_id,business_id,rev_stars,date,text,useful,funny,cool,name,neighborhood,address,city,state,postal_code,latitude,longitude,bus_stars,review_count,is_open,categories,weekday,text length,year
284676,771OWzbzelsEeSlx8QsfsQ,orMlHMLyHSldYgzfFTaeoA,4,2011-07-15,I really like this place. I have been numerous...,1,0,0,"""Eddie's House""",NaN,"""7042 E Indian School Rd""",Scottsdale,AZ,85251,33.495235,-111.929214,3.5,317,0,Seafood;American (New);Restaurants,4,509,2011
284680,Pj9YpaP0T7A_5S_PT8IpNg,OtVNeY6IfUh3CIOCzmfg3A,5,2012-08-15,Greasewood Flat is hugely popular with winter ...,4,3,2,"""Greasewood Flat""",NaN,"""27375 N Alma School Pkwy""",Scottsdale,AZ,85255,33.731797,-111.846131,4.0,123,0,Burgers;Restaurants;Hot Dogs;Nightlife;Bars,2,1463,2012
284687,7M1zIE6OzpySDlqLU6MnEg,is2RonWgyENNKOprcXQK6w,5,2011-11-04,3 times in 2 weeks should tell you thy this pl...,3,2,2,"""Osha Thai Cafe""",NaN,"""10953 N Frank Lloyd Wright Blvd, Ste 102""",Scottsdale,AZ,85259,33.585577,-111.834405,4.0,114,1,Salad;Restaurants;Soup;Thai,4,394,2011
284690,9uE0smG2bwgkI95RPj0lPQ,sKrlmbrZWCyLIgiMihCPqw,5,2015-11-16,I still love Hula's! Came here for a celebrat...,1,0,1,"""Hula's Modern Tiki Scottsdale""",NaN,"""7213 E 1st Ave""",Scottsdale,AZ,85251,33.493853,-111.925625,4.0,509,1,Hawaiian;Restaurants;Pan Asian,0,560,2015
284691,iN7T3vQoC3v_MPs2TzH5Zw,ghRXtBi--SW8uWQhrijBjw,4,2013-11-25,"In the daytime, Old Town is a cool destination...",3,0,3,"""Old Town Scottsdale""",NaN,"""""",Scottsdale,AZ,85251,33.498629,-111.922440,4.0,106,1,Shopping Centers;Transportation;Public Service...,0,1027,2013


In [28]:
print(f'Number of Users: {sdl_df.user_id.unique().shape[0]}')
print(f'Number of Restaurants: {sdl_df.business_id.unique().shape[0]}')
print(f'Number of Reviews: {sdl_df.text.unique().shape[0]}')

Number of Users: 3119
Number of Restaurants: 1210
Number of Reviews: 24738


In [29]:
stops = ['Restaurants', 'Food', 'Nightlife']

In [30]:
sdl_df['split_categories'] = sdl_df.categories.apply(lambda x: x.split(';'))

In [31]:
sdl_df.head()

,user_id,business_id,rev_stars,date,text,useful,funny,cool,name,neighborhood,address,city,state,postal_code,latitude,longitude,bus_stars,review_count,is_open,categories,weekday,text length,year,split_categories
284676,771OWzbzelsEeSlx8QsfsQ,orMlHMLyHSldYgzfFTaeoA,4,2011-07-15,I really like this place. I have been numerous...,1,0,0,"""Eddie's House""",NaN,"""7042 E Indian School Rd""",Scottsdale,AZ,85251,33.495235,-111.929214,3.5,317,0,Seafood;American (New);Restaurants,4,509,2011,"[Seafood, American (New), Restaurants]"
284680,Pj9YpaP0T7A_5S_PT8IpNg,OtVNeY6IfUh3CIOCzmfg3A,5,2012-08-15,Greasewood Flat is hugely popular with winter ...,4,3,2,"""Greasewood Flat""",NaN,"""27375 N Alma School Pkwy""",Scottsdale,AZ,85255,33.731797,-111.846131,4.0,123,0,Burgers;Restaurants;Hot Dogs;Nightlife;Bars,2,1463,2012,"[Burgers, Restaurants, Hot Dogs, Nightlife, Bars]"
284687,7M1zIE6OzpySDlqLU6MnEg,is2RonWgyENNKOprcXQK6w,5,2011-11-04,3 times in 2 weeks should tell you thy this pl...,3,2,2,"""Osha Thai Cafe""",NaN,"""10953 N Frank Lloyd Wright Blvd, Ste 102""",Scottsdale,AZ,85259,33.585577,-111.834405,4.0,114,1,Salad;Restaurants;Soup;Thai,4,394,2011,"[Salad, Restaurants, Soup, Thai]"
284690,9uE0smG2bwgkI95RPj0lPQ,sKrlmbrZWCyLIgiMihCPqw,5,2015-11-16,I still love Hula's! Came here for a celebrat...,1,0,1,"""Hula's Modern Tiki Scottsdale""",NaN,"""7213 E 1st Ave""",Scottsdale,AZ,85251,33.493853,-111.925625,4.0,509,1,Hawaiian;Restaurants;Pan Asian,0,560,2015,"[Hawaiian, Restaurants, Pan Asian]"
284691,iN7T3vQoC3v_MPs2TzH5Zw,ghRXtBi--SW8uWQhrijBjw,4,2013-11-25,"In the daytime, Old Town is a cool destination...",3,0,3,"""Old Town Scottsdale""",NaN,"""""",Scottsdale,AZ,85251,33.498629,-111.922440,4.0,106,1,Shopping Centers;Transportation;Public Service...,0,1027,2013,"[Shopping Centers, Transportation, Public Serv..."


In [32]:
sdl_df['split_categories'] = sdl_df.split_categories.apply(lambda x: [i for i in x if i not in stops])

In [33]:
sdl_df.head()

,user_id,business_id,rev_stars,date,text,useful,funny,cool,name,neighborhood,address,city,state,postal_code,latitude,longitude,bus_stars,review_count,is_open,categories,weekday,text length,year,split_categories
284676,771OWzbzelsEeSlx8QsfsQ,orMlHMLyHSldYgzfFTaeoA,4,2011-07-15,I really like this place. I have been numerous...,1,0,0,"""Eddie's House""",NaN,"""7042 E Indian School Rd""",Scottsdale,AZ,85251,33.495235,-111.929214,3.5,317,0,Seafood;American (New);Restaurants,4,509,2011,"[Seafood, American (New)]"
284680,Pj9YpaP0T7A_5S_PT8IpNg,OtVNeY6IfUh3CIOCzmfg3A,5,2012-08-15,Greasewood Flat is hugely popular with winter ...,4,3,2,"""Greasewood Flat""",NaN,"""27375 N Alma School Pkwy""",Scottsdale,AZ,85255,33.731797,-111.846131,4.0,123,0,Burgers;Restaurants;Hot Dogs;Nightlife;Bars,2,1463,2012,"[Burgers, Hot Dogs, Bars]"
284687,7M1zIE6OzpySDlqLU6MnEg,is2RonWgyENNKOprcXQK6w,5,2011-11-04,3 times in 2 weeks should tell you thy this pl...,3,2,2,"""Osha Thai Cafe""",NaN,"""10953 N Frank Lloyd Wright Blvd, Ste 102""",Scottsdale,AZ,85259,33.585577,-111.834405,4.0,114,1,Salad;Restaurants;Soup;Thai,4,394,2011,"[Salad, Soup, Thai]"
284690,9uE0smG2bwgkI95RPj0lPQ,sKrlmbrZWCyLIgiMihCPqw,5,2015-11-16,I still love Hula's! Came here for a celebrat...,1,0,1,"""Hula's Modern Tiki Scottsdale""",NaN,"""7213 E 1st Ave""",Scottsdale,AZ,85251,33.493853,-111.925625,4.0,509,1,Hawaiian;Restaurants;Pan Asian,0,560,2015,"[Hawaiian, Pan Asian]"
284691,iN7T3vQoC3v_MPs2TzH5Zw,ghRXtBi--SW8uWQhrijBjw,4,2013-11-25,"In the daytime, Old Town is a cool destination...",3,0,3,"""Old Town Scottsdale""",NaN,"""""",Scottsdale,AZ,85251,33.498629,-111.922440,4.0,106,1,Shopping Centers;Transportation;Public Service...,0,1027,2013,"[Shopping Centers, Transportation, Public Serv..."


In [34]:
sdl_df['string_categories'] = sdl_df.split_categories.apply(lambda x: ' '.join(x))

In [34]:
sdl_df.head()

,user_id,business_id,rev_stars,date,text,useful,funny,cool,name,neighborhood,address,city,state,postal_code,latitude,longitude,bus_stars,review_count,is_open,categories,weekday,text length,year,split_categories,string_categories
284676,771OWzbzelsEeSlx8QsfsQ,orMlHMLyHSldYgzfFTaeoA,4,2011-07-15,I really like this place. I have been numerous...,1,0,0,"""Eddie's House""",NaN,"""7042 E Indian School Rd""",Scottsdale,AZ,85251,33.495235,-111.929214,3.5,317,0,Seafood;American (New);Restaurants,4,509,2011,"[Seafood, American (New)]",Seafood American (New)
284680,Pj9YpaP0T7A_5S_PT8IpNg,OtVNeY6IfUh3CIOCzmfg3A,5,2012-08-15,Greasewood Flat is hugely popular with winter ...,4,3,2,"""Greasewood Flat""",NaN,"""27375 N Alma School Pkwy""",Scottsdale,AZ,85255,33.731797,-111.846131,4.0,123,0,Burgers;Restaurants;Hot Dogs;Nightlife;Bars,2,1463,2012,"[Burgers, Hot Dogs, Bars]",Burgers Hot Dogs Bars
284687,7M1zIE6OzpySDlqLU6MnEg,is2RonWgyENNKOprcXQK6w,5,2011-11-04,3 times in 2 weeks should tell you thy this pl...,3,2,2,"""Osha Thai Cafe""",NaN,"""10953 N Frank Lloyd Wright Blvd, Ste 102""",Scottsdale,AZ,85259,33.585577,-111.834405,4.0,114,1,Salad;Restaurants;Soup;Thai,4,394,2011,"[Salad, Soup, Thai]",Salad Soup Thai
284690,9uE0smG2bwgkI95RPj0lPQ,sKrlmbrZWCyLIgiMihCPqw,5,2015-11-16,I still love Hula's! Came here for a celebrat...,1,0,1,"""Hula's Modern Tiki Scottsdale""",NaN,"""7213 E 1st Ave""",Scottsdale,AZ,85251,33.493853,-111.925625,4.0,509,1,Hawaiian;Restaurants;Pan Asian,0,560,2015,"[Hawaiian, Pan Asian]",Hawaiian Pan Asian
284691,iN7T3vQoC3v_MPs2TzH5Zw,ghRXtBi--SW8uWQhrijBjw,4,2013-11-25,"In the daytime, Old Town is a cool destination...",3,0,3,"""Old Town Scottsdale""",NaN,"""""",Scottsdale,AZ,85251,33.498629,-111.922440,4.0,106,1,Shopping Centers;Transportation;Public Service...,0,1027,2013,"[Shopping Centers, Transportation, Public Serv...",Shopping Centers Transportation Public Service...


In [36]:
#sdl_df.to_csv('sdl_data.csv', encoding='utf-8')

#### Adding category columns

In [42]:
all_categories = list(sdl_df.categories.unique())

In [44]:
all_categories = ' '.join(all_categories)

In [46]:
all_categories = all_categories.split(';')

In [48]:
cats_df = pd.DataFrame(all_categories)

In [54]:
new_cats = list(cats_df.groupby(0).size().sort_values(ascending=False)[0:100].keys())

In [73]:
new_df = sdl_df.join(pd.DataFrame(columns=new_cats))

In [74]:
new_df = new_df.reset_index(drop=True)

In [75]:
new_df.head()

,user_id,business_id,rev_stars,date,text,useful,funny,cool,name,neighborhood,address,city,state,postal_code,latitude,longitude,bus_stars,review_count,is_open,categories,weekday,text length,year,split_categories,string_categories,Restaurants,Nightlife,Bars,Food,American (New),American (Traditional),Breakfast & Brunch,Sandwiches,Italian,Wine Bars,Restaurants Restaurants,Pizza,Sports Bars,Salad,Event Planning & Services,Seafood,Cafes,Coffee & Tea,Wine & Spirits,Burgers,Cocktail Bars,Lounges,Mexican,Beer,Steakhouses,Specialty Food,Bakeries,Sushi Bars,Vegetarian,Arts & Entertainment,Gluten-Free,Mediterranean,Juice Bars & Smoothies,Asian Fusion,Diners,Desserts,Japanese,Bars Restaurants,Delis,Pubs,Gastropubs,Restaurants Nightlife,Food Restaurants,Chicken Wings,Ice Cream & Frozen Yogurt,Caterers,Soup,Venues & Event Spaces,Bagels,Shopping,Vegan,American (Traditional) Restaurants,French,Barbeque,Latin American,Comfort Food,Active Life,Greek,Music Venues,American (New) Restaurants,Food Delivery Services,Restaurants Food,Nightlife Restaurants,Chinese,Beer Bar,Hotels & Travel,Middle Eastern,Thai,Tapas/Small Plates,Tapas Bars,Breakfast & Brunch Restaurants,Restaurants Bars,Local Flavor,Dance Clubs,Pizza Restaurants,Restaurants Sandwiches,Sandwiches Restaurants,Beer Gardens,Tex-Mex,Poke,Indian,Local Services,Restaurants Italian,Bars Nightlife,Korean,Southern,Food Breakfast & Brunch,American (New) Bars,Bars American (New),Bars Bars,Mexican Restaurants,Breweries,Restaurants Burgers,Italian Restaurants,Restaurants Japanese,Home & Garden,Golf,Vietnamese,Jazz & Blues,Restaurants Breakfast & Brunch
0,771OWzbzelsEeSlx8QsfsQ,orMlHMLyHSldYgzfFTaeoA,4,2011-07-15,I really like this place. I have been numerous...,1,0,0,"""Eddie's House""",NaN,"""7042 E Indian School Rd""",Scottsdale,AZ,85251,33.495235,-111.929214,3.5,317,0,Seafood;American (New);Restaurants,4,509,2011,"[Seafood, American (New)]",Seafood American (New),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pj9YpaP0T7A_5S_PT8IpNg,OtVNeY6IfUh3CIOCzmfg3A,5,2012-08-15,Greasewood Flat is hugely popular with winter ...,4,3,2,"""Greasewood Flat""",NaN,"""27375 N Alma School Pkwy""",Scottsdale,AZ,85255,33.731797,-111.846131,4.0,123,0,Burgers;Restaurants;Hot Dogs;Nightlife;Bars,2,1463,2012,"[Burgers, Hot Dogs, Bars]",Burgers Hot Dogs Bars,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7M1zIE6OzpySDlqLU6MnEg,is2RonWgyENNKOprcXQK6w,5,2011-11-04,3 times in 2 weeks should tell you thy this pl...,3,2,2,"""Osha Thai Cafe""",NaN,"""10953 N Frank Lloyd Wright Blvd, Ste 102""",Scottsdale,AZ,85259,33.585577,-111.834405,4.0,114,1,Salad;Restaurants;Soup;Thai,4,394,2011,"[Salad, Soup, Thai]",Salad Soup Thai,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9uE0smG2bwgkI95RPj0lPQ,sKrlmbrZWCyLIgiMihCPqw,5,2015-11-16,I still love Hula's! Came here for a celebrat...,1,0,1,"""Hula's Modern Tiki Scottsdale""",NaN,"""7213 E 1st Ave""",Scottsdale,AZ,85251,33.493853,-111.925625,4.0,50

In [133]:
for i in range(0, len(new_cats)):
    new_df[new_cats[i]] = 0

In [176]:
for i in new_cats:
    new_df[i] = new_df.apply(lambda x: 1 if i in x.split_categories else x[i], axis=1)

In [173]:
#new_df['Seafood'] = new_df.apply(lambda x: 1 if 'Seafood' in x.split_categories else x['Seafood'], axis=1)

In [177]:
new_df.head()

,user_id,business_id,rev_stars,date,text,useful,funny,cool,name,neighborhood,address,city,state,postal_code,latitude,longitude,bus_stars,review_count,is_open,categories,weekday,text length,year,split_categories,string_categories,Restaurants,Nightlife,Bars,Food,American (New),American (Traditional),Breakfast & Brunch,Sandwiches,Italian,Wine Bars,Restaurants Restaurants,Pizza,Sports Bars,Salad,Event Planning & Services,Seafood,Cafes,Coffee & Tea,Wine & Spirits,Burgers,Cocktail Bars,Lounges,Mexican,Beer,Steakhouses,Specialty Food,Bakeries,Sushi Bars,Vegetarian,Arts & Entertainment,Gluten-Free,Mediterranean,Juice Bars & Smoothies,Asian Fusion,Diners,Desserts,Japanese,Bars Restaurants,Delis,Pubs,Gastropubs,Restaurants Nightlife,Food Restaurants,Chicken Wings,Ice Cream & Frozen Yogurt,Caterers,Soup,Venues & Event Spaces,Bagels,Shopping,Vegan,American (Traditional) Restaurants,French,Barbeque,Latin American,Comfort Food,Active Life,Greek,Music Venues,American (New) Restaurants,Food Delivery Services,Restaurants Food,Nightlife Restaurants,Chinese,Beer Bar,Hotels & Travel,Middle Eastern,Thai,Tapas/Small Plates,Tapas Bars,Breakfast & Brunch Restaurants,Restaurants Bars,Local Flavor,Dance Clubs,Pizza Restaurants,Restaurants Sandwiches,Sandwiches Restaurants,Beer Gardens,Tex-Mex,Poke,Indian,Local Services,Restaurants Italian,Bars Nightlife,Korean,Southern,Food Breakfast & Brunch,American (New) Bars,Bars American (New),Bars Bars,Mexican Restaurants,Breweries,Restaurants Burgers,Italian Restaurants,Restaurants Japanese,Home & Garden,Golf,Vietnamese,Jazz & Blues,Restaurants Breakfast & Brunch
0,771OWzbzelsEeSlx8QsfsQ,orMlHMLyHSldYgzfFTaeoA,4,2011-07-15,I really like this place. I have been numerous...,1,0,0,"""Eddie's House""",NaN,"""7042 E Indian School Rd""",Scottsdale,AZ,85251,33.495235,-111.929214,3.5,317,0,Seafood;American (New);Restaurants,4,509,2011,"[Seafood, American (New)]",Seafood American (New),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Pj9YpaP0T7A_5S_PT8IpNg,OtVNeY6IfUh3CIOCzmfg3A,5,2012-08-15,Greasewood Flat is hugely popular with winter ...,4,3,2,"""Greasewood Flat""",NaN,"""27375 N Alma School Pkwy""",Scottsdale,AZ,85255,33.731797,-111.846131,4.0,123,0,Burgers;Restaurants;Hot Dogs;Nightlife;Bars,2,1463,2012,"[Burgers, Hot Dogs, Bars]",Burgers Hot Dogs Bars,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7M1zIE6OzpySDlqLU6MnEg,is2RonWgyENNKOprcXQK6w,5,2011-11-04,3 times in 2 weeks should tell you thy this pl...,3,2,2,"""Osha Thai Cafe""",NaN,"""10953 N Frank Lloyd Wright Blvd, Ste 102""",Scottsdale,AZ,85259,33.585577,-111.834405,4.0,114,1,Salad;Restaurants;Soup;Thai,4,394,2011,"[Salad, Soup, Thai]",Salad Soup Thai,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9uE0smG2bwgkI95RPj0lPQ,sKrlmbrZWCyLIgiMihCPqw,5,2015-11-16,I still love Hula's! Came here for a celebrat...,1,0,1,"""Hula's Modern Tiki Scottsdale""",NaN,"""7213 E 1st Ave""",Scottsdale,AZ,85251,33.493853,-111.925625,4.0,509,1,Hawaiian;Restaurants;Pan Asian,0,560,2015,"[Hawaiian, Pan Asian]",Hawaiian Pan Asian,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,iN7T3vQoC3v_MPs2TzH5Zw,ghRXtBi--SW8uWQhrijBjw,4,2013-11-25,"In the daytime, Old Town is a cool destination...",3,0,3,"""Old Town Scottsdale""",NaN,"""""",Scottsdale,AZ,85251,33.498629,-111.922440,4.0,106,1,Shopping Centers;Transportation;Public Service...,0,1027,2013,"[Shopping Centers, Transportation, Publi

In [210]:
new_df.shape

(24744, 125)

#### Adding attribute columns

In [178]:
df_att = pd.read_csv(f'{path}/external/yelp_business_attributes.csv')

In [179]:
df_att.head()

,business_id,AcceptsInsurance,ByAppointmentOnly,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,HairSpecializesIn_coloring,HairSpecializesIn_africanamerican,HairSpecializesIn_curly,HairSpecializesIn_perms,HairSpecializesIn_kids,HairSpecializesIn_extensions,HairSpecializesIn_asian,HairSpecializesIn_straightperms,RestaurantsPriceRange2,GoodForKids,WheelchairAccessible,BikeParking,Alcohol,HasTV,NoiseLevel,RestaurantsAttire,Music_dj,Music_background_music,Music_no_music,Music_karaoke,Music_live,Music_video,Music_jukebox,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_hipster,Ambience_divey,Ambience_touristy,Ambience_trendy,Ambience_upscale,Ambience_casual,RestaurantsGoodForGroups,Caters,WiFi,RestaurantsReservations,RestaurantsTakeOut,HappyHour,GoodForDancing,RestaurantsTableService,OutdoorSeating,RestaurantsDelivery,BestNights_monday,BestNights_tuesday,BestNights_friday,BestNights_wednesday,BestNights_thursday,BestNights_sunday,BestNights_saturday,GoodForMeal_dessert,GoodForMeal_latenight,GoodForMeal_lunch,GoodForMeal_dinner,GoodForMeal_breakfast,GoodForMeal_brunch,CoatCheck,Smoking,DriveThru,DogsAllowed,BusinessAcceptsBitcoin,Open24Hours,BYOBCorkage,BYOB,Corkage,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,AgesAllowed,RestaurantsCounterService
0,FYWN1wneV18bWNgQjJ2GNg,Na,Na,Na,True,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
1,He-G7vWjzVUysIKrfNbPUQ,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,True,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
2,8DShNS-LuFqpEWIp0HxijA,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,True,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
3,PfOCPjBrlQAnz__NXj9h_w,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,False,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
4,o9eMRCWt5PkpLDE0gOPtcQ,Na,Na,Na,Na,False,False,False,False,False,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,False,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na


In [180]:
scott_biz = list(new_df.business_id.unique())

In [181]:
#filter for Scottsdale restaurants
scott_att = df_att[df_att.business_id.isin(scott_biz)]

In [182]:
scott_att.head()

,business_id,AcceptsInsurance,ByAppointmentOnly,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,HairSpecializesIn_coloring,HairSpecializesIn_africanamerican,HairSpecializesIn_curly,HairSpecializesIn_perms,HairSpecializesIn_kids,HairSpecializesIn_extensions,HairSpecializesIn_asian,HairSpecializesIn_straightperms,RestaurantsPriceRange2,GoodForKids,WheelchairAccessible,BikeParking,Alcohol,HasTV,NoiseLevel,RestaurantsAttire,Music_dj,Music_background_music,Music_no_music,Music_karaoke,Music_live,Music_video,Music_jukebox,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_hipster,Ambience_divey,Ambience_touristy,Ambience_trendy,Ambience_upscale,Ambience_casual,RestaurantsGoodForGroups,Caters,WiFi,RestaurantsReservations,RestaurantsTakeOut,HappyHour,GoodForDancing,RestaurantsTableService,OutdoorSeating,RestaurantsDelivery,BestNights_monday,BestNights_tuesday,BestNights_friday,BestNights_wednesday,BestNights_thursday,BestNights_sunday,BestNights_saturday,GoodForMeal_dessert,GoodForMeal_latenight,GoodForMeal_lunch,GoodForMeal_dinner,GoodForMeal_breakfast,GoodForMeal_brunch,CoatCheck,Smoking,DriveThru,DogsAllowed,BusinessAcceptsBitcoin,Open24Hours,BYOBCorkage,BYOB,Corkage,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,AgesAllowed,RestaurantsCounterService
92,VdlPZg2NAu8t8GkdbPLecg,Na,Na,Na,Na,False,False,False,True,False,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,True,Na,Na,Na,Na,Na,Na,Na,False,False,False,True,False,False,Na,Na,Na,Na,Na,Na,Na,Na,Na,True,True,True,False,True,True,True,Na
196,Qv0OEziLJwyAqcgtrTsA4w,Na,Na,Na,Na,False,False,False,True,False,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,True,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
359,tCJGEQVqjELLYv3fPNX5Rw,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,True,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
365,rv9T1qGHbh9eXqFpKEPy_A,Na,Na,Na,Na,False,False,False,True,False,Na,Na,Na,Na,Na,Na,Na,Na,Na,True,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
507,0LKcSabd5Gd5oz-qrHTzZg,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,full_bar,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na


In [191]:
scott_att.shape

(1205, 82)

In [220]:
scott_att = scott_att.set_index('business_id')

In [221]:
scott_att.head()

,AcceptsInsurance,ByAppointmentOnly,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,HairSpecializesIn_coloring,HairSpecializesIn_africanamerican,HairSpecializesIn_curly,HairSpecializesIn_perms,HairSpecializesIn_kids,HairSpecializesIn_extensions,HairSpecializesIn_asian,HairSpecializesIn_straightperms,RestaurantsPriceRange2,GoodForKids,WheelchairAccessible,BikeParking,Alcohol,HasTV,NoiseLevel,RestaurantsAttire,Music_dj,Music_background_music,Music_no_music,Music_karaoke,Music_live,Music_video,Music_jukebox,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_hipster,Ambience_divey,Ambience_touristy,Ambience_trendy,Ambience_upscale,Ambience_casual,RestaurantsGoodForGroups,Caters,WiFi,RestaurantsReservations,RestaurantsTakeOut,HappyHour,GoodForDancing,RestaurantsTableService,OutdoorSeating,RestaurantsDelivery,BestNights_monday,BestNights_tuesday,BestNights_friday,BestNights_wednesday,BestNights_thursday,BestNights_sunday,BestNights_saturday,GoodForMeal_dessert,GoodForMeal_latenight,GoodForMeal_lunch,GoodForMeal_dinner,GoodForMeal_breakfast,GoodForMeal_brunch,CoatCheck,Smoking,DriveThru,DogsAllowed,BusinessAcceptsBitcoin,Open24Hours,BYOBCorkage,BYOB,Corkage,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,AgesAllowed,RestaurantsCounterService
business_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
VdlPZg2NAu8t8GkdbPLecg,Na,Na,Na,Na,False,False,False,True,False,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,True,Na,Na,Na,Na,Na,Na,Na,False,False,False,True,False,False,Na,Na,Na,Na,Na,Na,Na,Na,Na,True,True,True,False,True,True,True,Na
Qv0OEziLJwyAqcgtrTsA4w,Na,Na,Na,Na,False,False,False,True,False,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,True,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
tCJGEQVqjELLYv3fPNX5Rw,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,True,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
rv9T1qGHbh9eXqFpKEPy_A,Na,Na,Na,Na,False,False,False,True,False,Na,Na,Na,Na,Na,Na,Na,Na,Na,True,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
0LKcSabd5Gd5oz-qrHTzZg,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,full_bar,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na


In [225]:
scott_joined = new_df.join(scott_att, on=['business_id'], how='left')

In [226]:
scott_joined.head()

,user_id,business_id,rev_stars,date,text,useful,funny,cool,name,neighborhood,address,city,state,postal_code,latitude,longitude,bus_stars,review_count,is_open,categories,weekday,text length,year,split_categories,string_categories,Restaurants,Nightlife,Bars,Food,American (New),American (Traditional),Breakfast & Brunch,Sandwiches,Italian,Wine Bars,Restaurants Restaurants,Pizza,Sports Bars,Salad,Event Planning & Services,Seafood,Cafes,Coffee & Tea,Wine & Spirits,Burgers,Cocktail Bars,Lounges,Mexican,Beer,Steakhouses,Specialty Food,Bakeries,Sushi Bars,Vegetarian,Arts & Entertainment,Gluten-Free,Mediterranean,Juice Bars & Smoothies,Asian Fusion,Diners,Desserts,Japanese,Bars Restaurants,Delis,Pubs,Gastropubs,Restaurants Nightlife,Food Restaurants,Chicken Wings,Ice Cream & Frozen Yogurt,Caterers,Soup,Venues & Event Spaces,Bagels,Shopping,Vegan,American (Traditional) Restaurants,French,Barbeque,Latin American,Comfort Food,Active Life,Greek,Music Venues,American (New) Restaurants,Food Delivery Services,Restaurants Food,Nightlife Restaurants,Chinese,Beer Bar,Hotels & Travel,Middle Eastern,Thai,Tapas/Small Plates,Tapas Bars,Breakfast & Brunch Restaurants,Restaurants Bars,Local Flavor,Dance Clubs,Pizza Restaurants,...,Local Services,Restaurants Italian,Bars Nightlife,Korean,Southern,Food Breakfast & Brunch,American (New) Bars,Bars American (New),Bars Bars,Mexican Restaurants,Breweries,Restaurants Burgers,Italian Restaurants,Restaurants Japanese,Home & Garden,Golf,Vietnamese,Jazz & Blues,Restaurants Breakfast & Brunch,AcceptsInsurance,ByAppointmentOnly,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,HairSpecializesIn_coloring,HairSpecializesIn_africanamerican,HairSpecializesIn_curly,HairSpecializesIn_perms,HairSpecializesIn_kids,HairSpecializesIn_extensions,HairSpecializesIn_asian,HairSpecializesIn_straightperms,RestaurantsPriceRange2,GoodForKids,WheelchairAccessible,BikeParking,Alcohol,HasTV,NoiseLevel,RestaurantsAttire,Music_dj,Music_background_music,Music_no_music,Music_karaoke,Music_live,Music_video,Music_jukebox,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_hipster,Ambience_divey,Ambience_touristy,Ambience_trendy,Ambience_upscale,Ambience_casual,RestaurantsGoodForGroups,Caters,WiFi,RestaurantsReservations,RestaurantsTakeOut,HappyHour,GoodForDancing,RestaurantsTableService,OutdoorSeating,RestaurantsDelivery,BestNights_monday,BestNights_tuesday,BestNights_friday,BestNights_wednesday,BestNights_thursday,BestNights_sunday,BestNights_saturday,GoodForMeal_dessert,GoodForMeal_latenight,GoodForMeal_lunch,GoodForMeal_dinner,GoodForMeal_breakfast,GoodForMeal_brunch,CoatCheck,Smoking,DriveThru,DogsAllowed,BusinessAcceptsBitcoin,Open24Hours,BYOBCorkage,BYOB,Corkage,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,AgesAllowed,RestaurantsCounterService
0,771OWzbzelsEeSlx8QsfsQ,orMlHMLyHSldYgzfFTaeoA,4,2011-07-15,I really like this place. I have been numerous...,1,0,0,"""Eddie's House""",NaN,"""7042 E Indian School Rd""",Scottsdale,AZ,85251,33.495235,-111.929214,3.5,317,0,Seafood;American (New);Restaurants,4,509,2011,"[Seafood, American (New)]",Seafood American (New),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Na,Na,Na,Na,False,True,False,False,False,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,True,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
1,Pj9YpaP0T7A_5S_PT8IpNg,OtVNeY6IfUh3CIOCzmfg3A,5,2012-08-15,Greasewood Flat is hugely popular with winter ...,4,3,2,"""Greasewood Flat""",NaN,"""27375 N Alma School Pkwy""",Scottsdale,AZ,

In [227]:
scott_joined.shape

(24744, 206)

In [231]:
scott_joined = scott_joined.replace(to_replace='Na', value=np.nan)

In [233]:
scott_joined = scott_joined.replace(to_replace='False', value=0)

In [234]:
scott_joined = scott_joined.replace(to_replace='True', value=1)

In [236]:
scott_joined = scott_joined.fillna(0)

In [237]:
scott_joined.head()

,user_id,business_id,rev_stars,date,text,useful,funny,cool,name,neighborhood,address,city,state,postal_code,latitude,longitude,bus_stars,review_count,is_open,categories,weekday,text length,year,split_categories,string_categories,Restaurants,Nightlife,Bars,Food,American (New),American (Traditional),Breakfast & Brunch,Sandwiches,Italian,Wine Bars,Restaurants Restaurants,Pizza,Sports Bars,Salad,Event Planning & Services,Seafood,Cafes,Coffee & Tea,Wine & Spirits,Burgers,Cocktail Bars,Lounges,Mexican,Beer,Steakhouses,Specialty Food,Bakeries,Sushi Bars,Vegetarian,Arts & Entertainment,Gluten-Free,Mediterranean,Juice Bars & Smoothies,Asian Fusion,Diners,Desserts,Japanese,Bars Restaurants,Delis,Pubs,Gastropubs,Restaurants Nightlife,Food Restaurants,Chicken Wings,Ice Cream & Frozen Yogurt,Caterers,Soup,Venues & Event Spaces,Bagels,Shopping,Vegan,American (Traditional) Restaurants,French,Barbeque,Latin American,Comfort Food,Active Life,Greek,Music Venues,American (New) Restaurants,Food Delivery Services,Restaurants Food,Nightlife Restaurants,Chinese,Beer Bar,Hotels & Travel,Middle Eastern,Thai,Tapas/Small Plates,Tapas Bars,Breakfast & Brunch Restaurants,Restaurants Bars,Local Flavor,Dance Clubs,Pizza Restaurants,...,Local Services,Restaurants Italian,Bars Nightlife,Korean,Southern,Food Breakfast & Brunch,American (New) Bars,Bars American (New),Bars Bars,Mexican Restaurants,Breweries,Restaurants Burgers,Italian Restaurants,Restaurants Japanese,Home & Garden,Golf,Vietnamese,Jazz & Blues,Restaurants Breakfast & Brunch,AcceptsInsurance,ByAppointmentOnly,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,HairSpecializesIn_coloring,HairSpecializesIn_africanamerican,HairSpecializesIn_curly,HairSpecializesIn_perms,HairSpecializesIn_kids,HairSpecializesIn_extensions,HairSpecializesIn_asian,HairSpecializesIn_straightperms,RestaurantsPriceRange2,GoodForKids,WheelchairAccessible,BikeParking,Alcohol,HasTV,NoiseLevel,RestaurantsAttire,Music_dj,Music_background_music,Music_no_music,Music_karaoke,Music_live,Music_video,Music_jukebox,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_hipster,Ambience_divey,Ambience_touristy,Ambience_trendy,Ambience_upscale,Ambience_casual,RestaurantsGoodForGroups,Caters,WiFi,RestaurantsReservations,RestaurantsTakeOut,HappyHour,GoodForDancing,RestaurantsTableService,OutdoorSeating,RestaurantsDelivery,BestNights_monday,BestNights_tuesday,BestNights_friday,BestNights_wednesday,BestNights_thursday,BestNights_sunday,BestNights_saturday,GoodForMeal_dessert,GoodForMeal_latenight,GoodForMeal_lunch,GoodForMeal_dinner,GoodForMeal_breakfast,GoodForMeal_brunch,CoatCheck,Smoking,DriveThru,DogsAllowed,BusinessAcceptsBitcoin,Open24Hours,BYOBCorkage,BYOB,Corkage,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,AgesAllowed,RestaurantsCounterService
0,771OWzbzelsEeSlx8QsfsQ,orMlHMLyHSldYgzfFTaeoA,4,2011-07-15,I really like this place. I have been numerous...,1,0,0,"""Eddie's House""",0.0,"""7042 E Indian School Rd""",Scottsdale,AZ,85251,33.495235,-111.929214,3.5,317,0,Seafood;American (New);Restaurants,4,509,2011,"[Seafood, American (New)]",Seafood American (New),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Pj9YpaP0T7A_5S_PT8IpNg,OtVNeY6IfUh3CIOCzmfg3A,5,2012-08-15,Greasewood Flat is hugely popular with winter ...,4,3,2,"""Greasewoo

In [238]:
scott_joined.shape

(24744, 206)

In [240]:
#scott_joined['target'] = scott_joined.rev_stars.apply(lambda x: 1 if x >= 4 else 0)

#### Adding review topics columns

In [255]:
#import all libraries
import spacy
from nltk import FreqDist

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel, Phrases
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
import pyLDAvis.gensim
from gensim.test.utils import datapath

# Save model to disk.
temp_file = datapath("lda_scott_model")
#lda_model.save(temp_file)

# Load a potentially pretrained model from disk.
lda_model = LdaModel.load(temp_file)

In [244]:
topic_names = {0: u'service',
               1: u'sports bar',
               2: u'vegetarian, cafe',
               3: u'kid friendly',
               4: u'location',
               5: u'secret',
               6: u'sides, vietnamese',
               7: u'greek, hummus',
               8: u'food dishes',
               9: u'table, sit',
               10: u'golf',
               11: u'pizza, italian',
               12: u'dinner',
               13: u'big menue',
               14: u'sushi',
               15: u'bbq',
               16: u'dessert',
               17: u'french',
               18: u'healthy, salad',
               19: u'asian',
               20: u'quality, service',
               21: u'mall, food court',
               22: u'breakfast',
               23: u'burger & fries',
               24: u'wings, bbq',
               25: u'hotel',
               26: u'sushi',
               27: u'cheap, good, service',
               28: u'movie theater',
               29: u'comfort food',
               30: u'mexican',
               31: u'bars',
               32: u'italian',
               33: u'italian',
               34: u'coffee shop',
               35: u'lunch, sandwich',
               36: u'buffet',
               37: u'foreign, authentic',
               38: u'southern',
               39: u'service and seating',
               40: u'steak, dinner',
               41: u'happy hour, drinks',
               42: u'customer service',
               43: u'hard to find',
               44: u'tapas, appetizers',
               45: u'healthy',
               46: u'chinese',
               47: u'lunch',
               48: u'beer, wine, drinks',
               49: u'service'}

In [251]:
nlp = spacy.load('en', disable=['parser', 'ner'])

def lemmatization(texts, tags=['NOUN', 'ADJ', 'VERB']): # filter noun, verb, adjective
       output = []
       for sent in texts:
             doc = nlp(" ".join(sent)) 
            
             output.append([token.lemma_ for token in doc if token.pos_ in tags])
       return output

In [1]:
import os
import codecs
models = '/Users/dmitriykats/Documents/SpringBoard/Springboard/Capstone2/true_review/models/'
intermediate_directory = os.path.join(models, 'interim')
bigram_model_filepath = os.path.join(intermediate_directory, 'bigram_model_all')
trigram_model_filepath = os.path.join(intermediate_directory, 'trigram_model_all')
dictionary_filepath = os.path.join(intermediate_directory, 'dictionary_all')

In [263]:
#bigrams.save(os.path.join(bigram_model_filepath, 'bigram_model'))
bigrams = Phrases.load(os.path.join(bigram_model_filepath, 'bigram_model'))
trigrams = Phrases.load(os.path.join(trigram_model_filepath, 'trigram_model'))
dictionary = Dictionary.load(os.path.join(dictionary_filepath, 'dictionary'))

In [249]:
from nltk.corpus import stopwords
my_stops = ['The', 'the', 'www', 'yelp', 'scottsdale', 'http', 'com', 'yelp', 'http_www', 'food', 'restaurant', 'place', 'good', 'great']
stop_words = stopwords.words('english')
for word in my_stops:
    stop_words.append(word)
# function to remove stopwords

In [273]:
def lda_description(review_text, min_topic_freq=0.05):
    """
    accept the original text of a review and (1) parse it with spaCy,
    (2) apply text pre-proccessing steps, (3) create a bag-of-words
    representation, (4) create an LDA representation, and
    (5) print a sorted list of the top topics in the LDA representation
    """
    topic_list = []
    # remove unwanted characters, numbers and symbols
    processed_review = review_text.replace("[^a-zA-Z#]", " ")
    # remove short words (length < 3)
    processed_review = ' '.join([w for w in processed_review.split() if len(w)>2])
    # make entire processed lowercase
    sample_review = processed_review.lower()
    
    # remove stopwords from the processed
    sample_review = [word for word in sample_review.split() if word not in stop_words]
    tokenized_sample = sample_review
    tags=['NOUN', 'ADJ', 'VERB']
    sample_lemma = [token.lemma_ for token in nlp(' '.join(tokenized_sample)) if token.pos_ in tags]
    
    bigram_reviews_2 = bigrams[sample_lemma]
    
    trigram_reviews_2 = trigrams[bigram_reviews_2]
    
    # create a bag-of-words representation
    review_bow = dictionary.doc2bow(trigram_reviews_2)
    
    # create an LDA representation
    review_lda = lda_model[review_bow]
    sorted_review_lda = sorted(review_lda, key=lambda tub: tub[1], reverse=True)
    #print(sorted_review_lda)
    for topic_num, freq in sorted_review_lda:
        if freq < min_topic_freq:
            break
        #print('{:25}'.format(topic_names[topic_num], format('.2f')))
        topic_list.append(topic_names[topic_num])
    return topic_list

In [269]:
lda_description(scott_joined.text.iloc[0])

dinner                   
happy hour, drinks       
cheap, good, service     
lunch                    
buffet                   
healthy                  


In [274]:
scott_joined['review_topic'] = scott_joined.text.apply(lambda x: lda_description(x))

In [275]:
scott_joined.head()

,user_id,business_id,rev_stars,date,text,useful,funny,cool,name,neighborhood,address,city,state,postal_code,latitude,longitude,bus_stars,review_count,is_open,categories,weekday,text length,year,split_categories,string_categories,Restaurants,Nightlife,Bars,Food,American (New),American (Traditional),Breakfast & Brunch,Sandwiches,Italian,Wine Bars,Restaurants Restaurants,Pizza,Sports Bars,Salad,Event Planning & Services,Seafood,Cafes,Coffee & Tea,Wine & Spirits,Burgers,Cocktail Bars,Lounges,Mexican,Beer,Steakhouses,Specialty Food,Bakeries,Sushi Bars,Vegetarian,Arts & Entertainment,Gluten-Free,Mediterranean,Juice Bars & Smoothies,Asian Fusion,Diners,Desserts,Japanese,Bars Restaurants,Delis,Pubs,Gastropubs,Restaurants Nightlife,Food Restaurants,Chicken Wings,Ice Cream & Frozen Yogurt,Caterers,Soup,Venues & Event Spaces,Bagels,Shopping,Vegan,American (Traditional) Restaurants,French,Barbeque,Latin American,Comfort Food,Active Life,Greek,Music Venues,American (New) Restaurants,Food Delivery Services,Restaurants Food,Nightlife Restaurants,Chinese,Beer Bar,Hotels & Travel,Middle Eastern,Thai,Tapas/Small Plates,Tapas Bars,Breakfast & Brunch Restaurants,Restaurants Bars,Local Flavor,Dance Clubs,Pizza Restaurants,...,Restaurants Italian,Bars Nightlife,Korean,Southern,Food Breakfast & Brunch,American (New) Bars,Bars American (New),Bars Bars,Mexican Restaurants,Breweries,Restaurants Burgers,Italian Restaurants,Restaurants Japanese,Home & Garden,Golf,Vietnamese,Jazz & Blues,Restaurants Breakfast & Brunch,AcceptsInsurance,ByAppointmentOnly,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,HairSpecializesIn_coloring,HairSpecializesIn_africanamerican,HairSpecializesIn_curly,HairSpecializesIn_perms,HairSpecializesIn_kids,HairSpecializesIn_extensions,HairSpecializesIn_asian,HairSpecializesIn_straightperms,RestaurantsPriceRange2,GoodForKids,WheelchairAccessible,BikeParking,Alcohol,HasTV,NoiseLevel,RestaurantsAttire,Music_dj,Music_background_music,Music_no_music,Music_karaoke,Music_live,Music_video,Music_jukebox,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_hipster,Ambience_divey,Ambience_touristy,Ambience_trendy,Ambience_upscale,Ambience_casual,RestaurantsGoodForGroups,Caters,WiFi,RestaurantsReservations,RestaurantsTakeOut,HappyHour,GoodForDancing,RestaurantsTableService,OutdoorSeating,RestaurantsDelivery,BestNights_monday,BestNights_tuesday,BestNights_friday,BestNights_wednesday,BestNights_thursday,BestNights_sunday,BestNights_saturday,GoodForMeal_dessert,GoodForMeal_latenight,GoodForMeal_lunch,GoodForMeal_dinner,GoodForMeal_breakfast,GoodForMeal_brunch,CoatCheck,Smoking,DriveThru,DogsAllowed,BusinessAcceptsBitcoin,Open24Hours,BYOBCorkage,BYOB,Corkage,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,AgesAllowed,RestaurantsCounterService,review_topic
0,771OWzbzelsEeSlx8QsfsQ,orMlHMLyHSldYgzfFTaeoA,4,2011-07-15,I really like this place. I have been numerous...,1,0,0,"""Eddie's House""",0.0,"""7042 E Indian School Rd""",Scottsdale,AZ,85251,33.495235,-111.929214,3.5,317,0,Seafood;American (New);Restaurants,4,509,2011,"[Seafood, American (New)]",Seafood American (New),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[dinner, happy hour, drinks, cheap, good, serv..."
1,Pj9YpaP0T7A_5S_PT8IpNg,OtVNeY6IfUh3CIOCzmfg3A,5,2012-08-15,Greasewood Flat is h

In [287]:
topic_list = list(topic_names.values())

In [285]:
scott_joined = scott_joined.join(pd.DataFrame(columns=list(topic_names.values())))

In [286]:
scott_joined.head()

,user_id,business_id,rev_stars,date,text,useful,funny,cool,name,neighborhood,address,city,state,postal_code,latitude,longitude,bus_stars,review_count,is_open,categories,weekday,text length,year,split_categories,string_categories,Restaurants,Nightlife,Bars,Food,American (New),American (Traditional),Breakfast & Brunch,Sandwiches,Italian,Wine Bars,Restaurants Restaurants,Pizza,Sports Bars,Salad,Event Planning & Services,Seafood,Cafes,Coffee & Tea,Wine & Spirits,Burgers,Cocktail Bars,Lounges,Mexican,Beer,Steakhouses,Specialty Food,Bakeries,Sushi Bars,Vegetarian,Arts & Entertainment,Gluten-Free,Mediterranean,Juice Bars & Smoothies,Asian Fusion,Diners,Desserts,Japanese,Bars Restaurants,Delis,Pubs,Gastropubs,Restaurants Nightlife,Food Restaurants,Chicken Wings,Ice Cream & Frozen Yogurt,Caterers,Soup,Venues & Event Spaces,Bagels,Shopping,Vegan,American (Traditional) Restaurants,French,Barbeque,Latin American,Comfort Food,Active Life,Greek,Music Venues,American (New) Restaurants,Food Delivery Services,Restaurants Food,Nightlife Restaurants,Chinese,Beer Bar,Hotels & Travel,Middle Eastern,Thai,Tapas/Small Plates,Tapas Bars,Breakfast & Brunch Restaurants,Restaurants Bars,Local Flavor,Dance Clubs,Pizza Restaurants,...,Ambience_intimate,Ambience_classy,Ambience_hipster,Ambience_divey,Ambience_touristy,Ambience_trendy,Ambience_upscale,Ambience_casual,RestaurantsGoodForGroups,Caters,WiFi,RestaurantsReservations,RestaurantsTakeOut,HappyHour,GoodForDancing,RestaurantsTableService,OutdoorSeating,RestaurantsDelivery,BestNights_monday,BestNights_tuesday,BestNights_friday,BestNights_wednesday,BestNights_thursday,BestNights_sunday,BestNights_saturday,GoodForMeal_dessert,GoodForMeal_latenight,GoodForMeal_lunch,GoodForMeal_dinner,GoodForMeal_breakfast,GoodForMeal_brunch,CoatCheck,Smoking,DriveThru,DogsAllowed,BusinessAcceptsBitcoin,Open24Hours,BYOBCorkage,BYOB,Corkage,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,AgesAllowed,RestaurantsCounterService,review_topic,service,sports bar,"vegetarian, cafe",kid friendly,location,secret,"sides, vietnamese","greek, hummus",food dishes,"table, sit",golf,"pizza, italian",dinner,big menue,sushi,bbq,dessert,french,"healthy, salad",asian,"quality, service","mall, food court",breakfast,burger & fries,"wings, bbq",hotel,sushi,"cheap, good, service",movie theater,comfort food,mexican,bars,italian,italian,coffee shop,"lunch, sandwich",buffet,"foreign, authentic",southern,service and seating,"steak, dinner","happy hour, drinks",customer service,hard to find,"tapas, appetizers",healthy,chinese,lunch,"beer, wine, drinks",service
0,771OWzbzelsEeSlx8QsfsQ,orMlHMLyHSldYgzfFTaeoA,4,2011-07-15,I really like this place. I have been numerous...,1,0,0,"""Eddie's House""",0.0,"""7042 E Indian School Rd""",Scottsdale,AZ,85251,33.495235,-111.929214,3.5,317,0,Seafood;American (New);Restaurants,4,509,2011,"[Seafood, American (New)]",Seafood American (New),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[dinner, happy hour, drinks, cheap, good, serv...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pj9YpaP0T7A_5S_PT8IpNg,OtVNeY6IfUh3CIOCzmfg3A,5,2012-08-15,Greasewood Flat is hugely popular with winter ...,4,3,2,"""Greasewood Flat""",0.0,"""27375 N Alma School Pkwy""",Scottsdale,AZ,85255,33.731797,-111.846131,4.0,123,0,Burgers;Restaurants;Hot Dogs;Nightlife;Bars,2,1463,2012,"[Burgers, Hot Dogs, Bars]",Burgers Hot Dogs Bars,0,0,1,0,0,0,0,

In [291]:
for i in topic_list:
    scott_joined[i] = scott_joined.apply(lambda x: 1 if i in x.review_topic else x[i], axis=1)

In [293]:
scott_joined = scott_joined.fillna(0)

In [294]:
scott_joined.head()

,user_id,business_id,rev_stars,date,text,useful,funny,cool,name,neighborhood,address,city,state,postal_code,latitude,longitude,bus_stars,review_count,is_open,categories,weekday,text length,year,split_categories,string_categories,Restaurants,Nightlife,Bars,Food,American (New),American (Traditional),Breakfast & Brunch,Sandwiches,Italian,Wine Bars,Restaurants Restaurants,Pizza,Sports Bars,Salad,Event Planning & Services,Seafood,Cafes,Coffee & Tea,Wine & Spirits,Burgers,Cocktail Bars,Lounges,Mexican,Beer,Steakhouses,Specialty Food,Bakeries,Sushi Bars,Vegetarian,Arts & Entertainment,Gluten-Free,Mediterranean,Juice Bars & Smoothies,Asian Fusion,Diners,Desserts,Japanese,Bars Restaurants,Delis,Pubs,Gastropubs,Restaurants Nightlife,Food Restaurants,Chicken Wings,Ice Cream & Frozen Yogurt,Caterers,Soup,Venues & Event Spaces,Bagels,Shopping,Vegan,American (Traditional) Restaurants,French,Barbeque,Latin American,Comfort Food,Active Life,Greek,Music Venues,American (New) Restaurants,Food Delivery Services,Restaurants Food,Nightlife Restaurants,Chinese,Beer Bar,Hotels & Travel,Middle Eastern,Thai,Tapas/Small Plates,Tapas Bars,Breakfast & Brunch Restaurants,Restaurants Bars,Local Flavor,Dance Clubs,Pizza Restaurants,...,Ambience_intimate,Ambience_classy,Ambience_hipster,Ambience_divey,Ambience_touristy,Ambience_trendy,Ambience_upscale,Ambience_casual,RestaurantsGoodForGroups,Caters,WiFi,RestaurantsReservations,RestaurantsTakeOut,HappyHour,GoodForDancing,RestaurantsTableService,OutdoorSeating,RestaurantsDelivery,BestNights_monday,BestNights_tuesday,BestNights_friday,BestNights_wednesday,BestNights_thursday,BestNights_sunday,BestNights_saturday,GoodForMeal_dessert,GoodForMeal_latenight,GoodForMeal_lunch,GoodForMeal_dinner,GoodForMeal_breakfast,GoodForMeal_brunch,CoatCheck,Smoking,DriveThru,DogsAllowed,BusinessAcceptsBitcoin,Open24Hours,BYOBCorkage,BYOB,Corkage,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,AgesAllowed,RestaurantsCounterService,review_topic,service,sports bar,"vegetarian, cafe",kid friendly,location,secret,"sides, vietnamese","greek, hummus",food dishes,"table, sit",golf,"pizza, italian",dinner,big menue,sushi,bbq,dessert,french,"healthy, salad",asian,"quality, service","mall, food court",breakfast,burger & fries,"wings, bbq",hotel,sushi,"cheap, good, service",movie theater,comfort food,mexican,bars,italian,italian,coffee shop,"lunch, sandwich",buffet,"foreign, authentic",southern,service and seating,"steak, dinner","happy hour, drinks",customer service,hard to find,"tapas, appetizers",healthy,chinese,lunch,"beer, wine, drinks",service
0,771OWzbzelsEeSlx8QsfsQ,orMlHMLyHSldYgzfFTaeoA,4,2011-07-15,I really like this place. I have been numerous...,1,0,0,"""Eddie's House""",0.0,"""7042 E Indian School Rd""",Scottsdale,AZ,85251,33.495235,-111.929214,3.5,317,0,Seafood;American (New);Restaurants,4,509,2011,"[Seafood, American (New)]",Seafood American (New),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[dinner, happy hour, drinks, cheap, good, serv...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,Pj9YpaP0T7A_5S_PT8IpNg,OtVNeY6IfUh3CIOCzmfg3A,5,2012-08-15,Greasewood Flat is hugely popular with winter ...,4,3,2,"""Greasewood Flat""",0.0,"""27375 N Alma School Pkwy""",Scottsdale,AZ,85255,33.731797,-111.846131,4.0,123,0,Burgers;Restaurants;Hot Dogs;Nightlife;Bars,2,1463,2012,"[Burgers, Hot Dogs, Bars]",Burgers Hot Dogs Bars,0,0,1,0,0,0,0,

#### Adding a target column

In [298]:
scott_joined['target'] = scott_joined.rev_stars.apply(lambda x: 1 if x >= 4 else 0)

In [299]:
scott_joined.head()

,user_id,business_id,rev_stars,date,text,useful,funny,cool,name,neighborhood,address,city,state,postal_code,latitude,longitude,bus_stars,review_count,is_open,categories,weekday,text length,year,split_categories,string_categories,Restaurants,Nightlife,Bars,Food,American (New),American (Traditional),Breakfast & Brunch,Sandwiches,Italian,Wine Bars,Restaurants Restaurants,Pizza,Sports Bars,Salad,Event Planning & Services,Seafood,Cafes,Coffee & Tea,Wine & Spirits,Burgers,Cocktail Bars,Lounges,Mexican,Beer,Steakhouses,Specialty Food,Bakeries,Sushi Bars,Vegetarian,Arts & Entertainment,Gluten-Free,Mediterranean,Juice Bars & Smoothies,Asian Fusion,Diners,Desserts,Japanese,Bars Restaurants,Delis,Pubs,Gastropubs,Restaurants Nightlife,Food Restaurants,Chicken Wings,Ice Cream & Frozen Yogurt,Caterers,Soup,Venues & Event Spaces,Bagels,Shopping,Vegan,American (Traditional) Restaurants,French,Barbeque,Latin American,Comfort Food,Active Life,Greek,Music Venues,American (New) Restaurants,Food Delivery Services,Restaurants Food,Nightlife Restaurants,Chinese,Beer Bar,Hotels & Travel,Middle Eastern,Thai,Tapas/Small Plates,Tapas Bars,Breakfast & Brunch Restaurants,Restaurants Bars,Local Flavor,Dance Clubs,Pizza Restaurants,...,Ambience_classy,Ambience_hipster,Ambience_divey,Ambience_touristy,Ambience_trendy,Ambience_upscale,Ambience_casual,RestaurantsGoodForGroups,Caters,WiFi,RestaurantsReservations,RestaurantsTakeOut,HappyHour,GoodForDancing,RestaurantsTableService,OutdoorSeating,RestaurantsDelivery,BestNights_monday,BestNights_tuesday,BestNights_friday,BestNights_wednesday,BestNights_thursday,BestNights_sunday,BestNights_saturday,GoodForMeal_dessert,GoodForMeal_latenight,GoodForMeal_lunch,GoodForMeal_dinner,GoodForMeal_breakfast,GoodForMeal_brunch,CoatCheck,Smoking,DriveThru,DogsAllowed,BusinessAcceptsBitcoin,Open24Hours,BYOBCorkage,BYOB,Corkage,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,AgesAllowed,RestaurantsCounterService,review_topic,service,sports bar,"vegetarian, cafe",kid friendly,location,secret,"sides, vietnamese","greek, hummus",food dishes,"table, sit",golf,"pizza, italian",dinner,big menue,sushi,bbq,dessert,french,"healthy, salad",asian,"quality, service","mall, food court",breakfast,burger & fries,"wings, bbq",hotel,sushi,"cheap, good, service",movie theater,comfort food,mexican,bars,italian,italian,coffee shop,"lunch, sandwich",buffet,"foreign, authentic",southern,service and seating,"steak, dinner","happy hour, drinks",customer service,hard to find,"tapas, appetizers",healthy,chinese,lunch,"beer, wine, drinks",service,target
0,771OWzbzelsEeSlx8QsfsQ,orMlHMLyHSldYgzfFTaeoA,4,2011-07-15,I really like this place. I have been numerous...,1,0,0,"""Eddie's House""",0.0,"""7042 E Indian School Rd""",Scottsdale,AZ,85251,33.495235,-111.929214,3.5,317,0,Seafood;American (New);Restaurants,4,509,2011,"[Seafood, American (New)]",Seafood American (New),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[dinner, happy hour, drinks, cheap, good, serv...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1
1,Pj9YpaP0T7A_5S_PT8IpNg,OtVNeY6IfUh3CIOCzmfg3A,5,2012-08-15,Greasewood Flat is hugely popular with winter ...,4,3,2,"""Greasewood Flat""",0.0,"""27375 N Alma School Pkwy""",Scottsdale,AZ,85255,33.731797,-111.846131,4.0,123,0,Burgers;Restaurants;Hot Dogs;Nightlife;Bars,2,1463,2012,"[Burgers, Hot Dogs, Bars]",Burgers Hot Dogs Bars,0,0,1,0,0,0,0,0,0,0,0,0,0,0

In [301]:
process_path = '/Users/dmitriykats/Documents/SpringBoard/Springboard/Capstone2/true_review/data/'

In [302]:
scott_joined.to_csv(f'{process_path}processed/scottsdale_modeling.csv', encoding='utf-8')